Hello, welcome to our `boomdiff` package tutorial series. 

The `boomdiff` is a package implementing forward-mode auto differentiations and graident-based optimizations of user-specified or pre-set objective functions. The organization of `boomdiff` package is highly modularized, with three major modules: 

1. The `boomdiff.AD` class, as the core functionality of `boomdiff` package, provides the interface to create, operate variables and track their gradients. Such functionality is realized through the `AD` instances or `AD` instances array data structures. We will walk thorugh this in tutorial section 1.

2. The `boomdiff.optimize` module includes optimization algorithms based on the gradients of loss functions, user-defined or pre-set. We will illustrate the usage in tutorial section 2.

3. The `boomdiff.loss_function` module includes some pre-set loss functions, like mean squared error (MSE), for users' convenience. We will illustrate the usage in tutorial section 2

Following the basic tutorials (if you are proficient at progamming, you can probably skip the basic tutorials), two general pedagogical examples will be given 

- A linear regression model in tutorial section 3 

- A logistic regression model in tutorial section 4

- A simple neural network model in section 5. (Yes! we are proud that `boomdiff` can be used as a deep learning framework! Although there are still lots of things to do regarding performance.)

These two examples will include most features, show you the usability and power of the `boomdiff` package, and you can probably understand the basic logics to use `boomdiff`: 

$$\text{Create variables} \to \text{Construct models and define target functions} \to \text{Optimization}$$

Then, you can construct your own models with `boomdiff` and solve real-life optimization problems!

## 1.1 Create a AD instance as a scalar variable

To start, make sure you have followed the [installation tutorial](https://github.com/team-boomeraang/cs107-FinalProject/blob/master/README.md#installation-of-boomdiff) and successfully installed the `boomdiff` package. If this is the case, we can import the package: 

In [1]:
from boomdiff import AD

The instantiation of a single variable with AD is quite intuitive, we can simply call:

In [2]:
a = AD(10., {'a': 1.0})

Then a variable called `a` is created, it is an AD class instance. There are two arguments should be put in such instantiation: value and partial derivative dictionary. The two property can be called by attributes `func_val` and `partial_dict`:

In [3]:
print(a.func_val)
print(a.partial_dict)

10.0
{'a': 1.0}


Here the value is `10`, which means the variable `a` itself is at 10. And the partial derivative dictionary `{'a': 1.0}` means, the variable's partial derivative to the name `'a'` is 1.0. **Name string** is one key property used to tracking the gradient in the multi-variable case. Here, as we haven't put any operations to such a variable, you can simple view the string `'a'` as the name of the variable, the derivative to itself should mostly be 1 (you can set it to other values as a seed vector).

Based on such motivation, we also support create a varible in a simpler manner:  

In [4]:
b = AD(7,'b')

In [5]:
print(b.func_val)
print(b.partial_dict)

7
{'b': 1.0}


As above, you can only give the value and a name string, the partial derivative to itself will be set to 1.0 by default. Now you have another variable `b`.

> Note: When dealing with multi-variable cases, make sure the name strings of your different variables are different! And the best practice is making the name of variables and their name string consistent, i.e. do `a = AD(10, 'a')` instead of `a = AD(10, 'b')`.

And, if you are super lazy and will only work with a single variable, so the name string is not quite meaningful for you. We also support such syntax: 

In [6]:
x1 = AD(5.0)

In [7]:
print(x1.func_val)
print(x1.partial_dict)

5.0
{'x1': 1}


As shown, you can only put an value and the name string is set to `'x1'` by default. Now you have another variable `x1`. You will see the power of the name string in following operations. 

## 1.2 Apply operations to AD instances and track gradient 

With the three variables `a`, `b`, `x1` we created above, we can do some operations. Let's start with an simple case: `f = 2*a + 3*b - 4*x1`. For this case, we can simply calculate by hand that:

$$f=21,\quad \frac{\partial f}{\partial a} = 2, \quad \frac{\partial f}{\partial b} = 3, \quad \frac{\partial f}{\partial x1} = -4$$

This calcualation can also be done quite intuitively in `boomdiff`:

In [8]:
f = 2*a + 3*b - 4*x1

In [9]:
print("Function value: ", f.func_val)
print("Partial derivatives: ", f.partial_dict)

Function value:  21.0
Partial derivatives:  {'a': 2.0, 'b': 3.0, 'x1': -4}


The object `f` is still an AD instance. Besides the function value, the name strings in its `partial_dict` atribute clearly show the gradients relation. Now you can see why the name string is important. Furthermore, as the `f` is still an AD instance, you can continue to apply operations on it and extend the computational graph, the gradients tape will still hold:

In [10]:
f2 = f**2 + AD.sin(a)/AD.exp(b) + AD.log(x1)

In [11]:
print("Function value: ", f2.func_val)
print("Partial derivatives: ", f2.partial_dict)

Function value:  442.6089418293942
Partial derivatives:  {'a': 83.99923486580482, 'b': 126.0004960830399, 'x1': -167.8}


If you are annoyed by the long float expression like me, you can control the decimal rounded length by a helper method `round`:

In [12]:
print("Rounded Function value: ", f2.round(1).func_val)
print("Rounded Partial derivatives: ", f2.round(1).partial_dict)

Rounded Function value:  442.6
Rounded Partial derivatives:  {'a': 84.0, 'b': 126.0, 'x1': -167.8}


Currently, the `boomdiff` has alrealy support a huge amount of basic operations, functions and helper methods. For a complete API list and descriptions, see [AD API](https://github.com/team-boomeraang/cs107-FinalProject/blob/master/docs/documentation.md#autodiff).

If you are done with the tracked operations and only want the function values, we provide a simple method called `value` to detach, it will return simple number values:

In [13]:
f2.round(4).value()

442.6089

## 1.3 Create AD instances arrays as variable arrays

Above, we demonstrated how to create and operate scalar varaibles in `boomdiff`. However, sometimes the number of parameters in a real-life model is quite large, it might be exhausting to create them one by one. Based on such motivation, we develop the following tools to create AD instances arrays as variable arrays. You can create a bunch of parameters with few lines.

At the moment, `numpy>=1.19` is needed to support all desired features, we import `numpy`:

In [14]:
import numpy as np

In [15]:
np.random.seed(14) # For reproducibility

Now let's say we want a `2*2` parameters matrix `w1`, we can make it by two lines:

1. Create an array with `numpy`, with size equal `2*2`

In [16]:
w1_np = np.array([[1.,2.],[3.,4.]])

2. Convert it to AD instances arrays with `AD.from_array()` method:

In [17]:
w1 = AD.from_array(w1_np, prefix='w1')

In [18]:
print(w1)

[[1.0 ({'w1_0_0': 1.0}) 2.0 ({'w1_0_1': 1.0})]
 [3.0 ({'w1_1_0': 1.0}) 4.0 ({'w1_1_1': 1.0})]]


Now we have the parameters matrix `w1`. You can see that object `w1` is an array with all elements are AD instances. The value of the elements are determined by the `w1_np` array, and the name string is `prefix_i_j`, `i` and `j` are the row and column index in the matrix, so each element will have different name strings. All derivatives here are set to 1.0 by default.

You can simple convert `w1` back by `AD.to_array()` method:

In [19]:
AD.to_array(w1)

array([[1., 2.],
       [3., 4.]])

## 1.4 Apply operations to AD instances array and track gradient

All operations mentioned in section 1.2 will still work for AD instances arrays, either element-wise or broadcast. For example:

In [20]:
w1**2

array([[1.0 ({'w1_0_0': 2.0}), 4.0 ({'w1_0_1': 4.0})],
       [9.0 ({'w1_1_0': 6.0}), 16.0 ({'w1_1_1': 8.0})]], dtype=object)

In [21]:
w1/w1

array([[1.0 ({'w1_0_0': 0.0}), 1.0 ({'w1_0_1': 0.0})],
       [1.0 ({'w1_1_0': 0.0}), 1.0 ({'w1_1_1': 0.0})]], dtype=object)

In [22]:
AD.log(w1)

array([[0.0 ({'w1_0_0': 1.0}), 0.6931471805599453 ({'w1_0_1': 0.5})],
       [1.0986122886681098 ({'w1_1_0': 0.3333333333333333}),
        1.3862943611198906 ({'w1_1_1': 0.25})]], dtype=object)

In [23]:
AD.tanh(w1)

array([[0.7615941559557649 ({'w1_0_0': 0.4199743416140261}),
        0.9640275800758169 ({'w1_0_1': 0.07065082485316447})],
       [0.9950547536867305 ({'w1_1_0': 0.009866037165440192}),
        0.999329299739067 ({'w1_1_1': 0.001340950683025897})]],
      dtype=object)

---

Besides, there are some array-specific operations, like `AD.sum()`, `AD.mean()`, `AD.dot()`. For example, we can define Frobinius norm of `w1` by one line, and the gradients are tracked

In [24]:
AD.sum(w1**2)

30.0 ({'w1_0_0': 2.0, 'w1_0_1': 4.0, 'w1_1_0': 6.0, 'w1_1_1': 8.0})

---

We support matrix operations between a `numpy` array and an AD instances array:

In [25]:
A = np.random.randint(0,5,size=[2,1])

In [35]:
A

array([[3],
       [0]])

In [26]:
AD.dot(w1,A)

array([[3.0 ({'w1_0_0': 3.0, 'w1_0_1': 0.0})],
       [9.0 ({'w1_1_0': 3.0, 'w1_1_1': 0.0})]], dtype=object)

In [27]:
w1@A

array([[3.0 ({'w1_0_0': 3.0, 'w1_0_1': 0.0})],
       [9.0 ({'w1_1_0': 3.0, 'w1_1_1': 0.0})]], dtype=object)

In [28]:
B = np.random.randint(0,5,size=[2,2])

In [36]:
B

array([[4, 1],
       [2, 4]])

In [29]:
w1+B

array([[5.0 ({'w1_0_0': 1.0}), 3.0 ({'w1_0_1': 1.0})],
       [5.0 ({'w1_1_0': 1.0}), 8.0 ({'w1_1_1': 1.0})]], dtype=object)

---

We support matrix operations between two AD instances arrays:

In [30]:
w2 = AD.from_array(np.random.randint(0,5,size=[2,1]), prefix="w2")

In [34]:
w2

array([[2 ({'w2_0_0': 1.0})],
       [0 ({'w2_1_0': 1.0})]], dtype=object)

In [31]:
w1@w2

array([[2.0 ({'w1_0_0': 2.0, 'w2_0_0': 1.0, 'w1_0_1': 0.0, 'w2_1_0': 2.0})],
       [6.0 ({'w1_1_0': 2.0, 'w2_0_0': 3.0, 'w1_1_1': 0.0, 'w2_1_0': 4.0})]],
      dtype=object)

In [32]:
w3 = AD.from_array(np.random.randint(0,5,size=[2,2]), prefix="w3")

In [38]:
w3

array([[0 ({'w3_0_0': 1.0}), 2 ({'w3_0_1': 1.0})],
       [1 ({'w3_1_0': 1.0}), 3 ({'w3_1_1': 1.0})]], dtype=object)

In [37]:
w1-w3

array([[1.0 ({'w1_0_0': 1.0, 'w3_0_0': -1.0}),
        0.0 ({'w1_0_1': 1.0, 'w3_0_1': -1.0})],
       [2.0 ({'w1_1_0': 1.0, 'w3_1_0': -1.0}),
        1.0 ({'w1_1_1': 1.0, 'w3_1_1': -1.0})]], dtype=object)

---

We support operations between an AD instance and an AD instances arrays, in a broadcast manner:

In [39]:
a

10.0 ({'a': 1.0})

In [40]:
w1+a

array([[11.0 ({'w1_0_0': 1.0, 'a': 1.0}),
        12.0 ({'w1_0_1': 1.0, 'a': 1.0})],
       [13.0 ({'w1_1_0': 1.0, 'a': 1.0}),
        14.0 ({'w1_1_1': 1.0, 'a': 1.0})]], dtype=object)

All the operations will output another AD instances array or an single AD instance, keeping the gradients tracked.

With all the tools we learned here, we can smoothly move to the optimization tutorial! 